In [2]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [3]:
# Cargamos el dataset
data = load_breast_cancer()

# Convertimos en DataFrame
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

In [4]:
# Dividimos el dataset en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Mostrar forma de los datos
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train distrib:", y_train.sum(), "positivos,", len(y_train) - y_train.sum(), "negativos")


X_train shape: (455, 30)
X_test shape: (114, 30)
y_train distrib: 285 positivos, 170 negativos


### Nota:
Cuando usas stratify=y, le estás diciendo a train_test_split que mantenga la misma proporción de clases (0s y 1s) en los subconjuntos de entrenamiento y prueba.

### Crear el Pipeline básico con un solo modelo

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [6]:
# Creamos el Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Entrenamos el pipeline
pipeline.fit(X_train, y_train)

# Predecimos
y_pred = pipeline.predict(X_test)

# Evaluamos
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        42
           1       0.99      0.99      0.99        72

    accuracy                           0.98       114
   macro avg       0.98      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



### Probar múltiples modelos con Pipeline

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [11]:
# Creamos el diccionario de modelos
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC()
}

# Probar con cada modelo
for model_name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    
    # Predicciones en test
    y_pred_test = pipeline.predict(X_test)
    
    # Predicciones en train
    y_pred_train = pipeline.predict(X_train)
    
    print(f"\n=== {model_name} ===")
    print("---- Train ----")
    print(classification_report(y_train, y_pred_train))
    
    print("---- Test ----")
    print(classification_report(y_test, y_pred_test))


=== Logistic Regression ===
---- Train ----
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       170
           1       0.99      1.00      0.99       285

    accuracy                           0.99       455
   macro avg       0.99      0.99      0.99       455
weighted avg       0.99      0.99      0.99       455

---- Test ----
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        42
           1       0.99      0.99      0.99        72

    accuracy                           0.98       114
   macro avg       0.98      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114


=== Random Forest ===
---- Train ----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       170
           1       1.00      1.00      1.00       285

    accuracy                           1.00       455
   macro avg       1.00      

### GridSearchCV para LogisticRegression y SVM

##### Objetivo
Encontrar los mejores hiperparámetros para cada modelo.

Usar Pipeline para evitar data leakage.

Evaluar el modelo optimizado.


In [12]:
# Pipeline para para SVM
pipe_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])


param_grid_svm = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__gamma': ['scale', 'auto'] # sólo aplica para rbf
}

# GridSearch
grid_svm = GridSearchCV(pipe_svm, param_grid_svm, cv=5)
grid_svm.fit(X_train, y_train)

print('\nMejor modelo SVM:')
print('Mejores hiperparámetros:', grid_svm.best_params_)
print('Score en test (Accuracy):', grid_svm.score(X_test, y_test))


Mejor modelo SVM:
Mejores hiperparámetros: {'classifier__C': 0.1, 'classifier__gamma': 'scale', 'classifier__kernel': 'linear'}
Score en test (Accuracy): 0.9824561403508771


In [13]:
import joblib

# Guardar el mejor modelo (pipeline ya entrenado)
joblib.dump(grid_svm.best_estimator_, 'cancer_model.pkl')

['cancer_model.pkl']

### Crear una API con FastAPI que use el .pkl
Esto te permitirá:

Hacer predicciones enviando datos por HTTP (desde otro sistema, app, etc.).

Tener tu modelo “en línea” para pruebas o integración.

#### Estructura básica del archivo app.py
Crea un archivo llamado app.py en la misma carpeta donde está el .pkl.

In [14]:
X_test.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
256,19.55,28.77,133.60,1207.0,0.09260,0.20630,0.17840,0.11440,0.1893,0.06232,...,25.05,36.27,178.60,1926.0,0.1281,0.53290,0.4251,0.19410,0.2818,0.10050
428,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,0.06148,...,11.68,20.29,74.35,421.1,0.1030,0.06219,0.0458,0.04044,0.2383,0.07083
501,13.82,24.49,92.33,595.9,0.11620,0.16810,0.13570,0.06759,0.2275,0.07237,...,16.01,32.94,106.00,788.0,0.1794,0.39660,0.3381,0.15210,0.3651,0.11830
363,16.50,18.29,106.60,838.1,0.09686,0.08468,0.05862,0.04835,0.1495,0.05593,...,18.13,25.45,117.20,1009.0,0.1338,0.16790,0.1663,0.09123,0.2394,0.06469
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.45,26.40,166.10,2027.0,0.1410,0.21130,0.4107,0.22160,0.2060,0.07115


In [47]:
X_train.head(13)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
546,10.32,16.35,65.31,324.9,0.09434,0.04994,0.01012,0.005495,0.1885,0.06201,...,11.250,21.77,71.12,384.9,0.12850,0.08842,0.04384,0.02381,0.2681,0.07399
432,20.18,19.54,133.80,1250.0,0.11330,0.14890,0.21330,0.125900,0.1724,0.06053,...,22.030,25.07,146.00,1479.0,0.16650,0.29420,0.53080,0.21730,0.3032,0.08075
174,10.66,15.15,67.49,349.6,0.08792,0.04302,0.00000,0.000000,0.1928,0.05975,...,11.540,19.20,73.20,408.3,0.10760,0.06791,0.00000,0.00000,0.2710,0.06164
221,13.56,13.90,88.59,561.3,0.10510,0.11920,0.07860,0.044510,0.1962,0.06303,...,14.980,17.13,101.10,686.6,0.13760,0.26980,0.25770,0.09090,0.3065,0.08177
289,11.37,18.89,72.17,396.0,0.08713,0.05008,0.02399,0.021730,0.2013,0.05955,...,12.360,26.14,79.29,459.3,0.11180,0.09708,0.07529,0.06203,0.3267,0.06994
147,14.95,18.77,97.84,689.5,0.08138,0.11670,0.09050,0.035620,0.1744,0.06493,...,16.250,25.47,107.10,809.7,0.09970,0.25210,0.25000,0.08405,0.2852,0.09218
462,14.40,26.99,92.25,646.1,0.06995,0.05223,0.03476,0.017370,0.1707,0.05433,...,15.400,31.98,100.40,734.6,0.10170,0.14600,0.14720,0.05563,0.2345,0.06464
192,9.72,18.22,60.73,288.1,0.06950,0.02344,0.00000,0.000000,0.1653,0.06447,...,9.968,20.83,62.25,303.8,0.07117,0.02729,0.00000,0.00000,0.1909,0.06559
116,8.95,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.023080,0.1305,0.07163,...,9.414,17.07,63.34,270.0,0.11790,0.18790,0.15440,0.03846,0.1652,0.07722
513,14.58,13.66,94.29,658.8,0.09832,0.08918,0.08222,0.043490,0.1739,0.05640,...,16.760,17.24,108.50,862.0,0.12230,0.19280,0.24920,0.09186,0.2626,0.07048


In [43]:
X_train.to_csv('C:/Users/mmora/Jupyter_Notebooks/Pipelines.train.csv')

In [44]:
X_train.to_csv('train.csv')

In [45]:
y_train

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,

In [40]:
X_test['fractal dimension error']

256    0.005252
428    0.002377
501    0.006995
363    0.002001
564    0.004239
         ...   
95     0.004411
128    0.004984
257    0.008660
228    0.003407
488    0.003213
Name: fractal dimension error, Length: 114, dtype: float64

In [ ]:
df = pd.read_csv(file)